In [ ]:
from google.colab import drive                          
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os 
os.chdir("/content/drive/MyDrive/")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('new_card_transactions.csv')

In [ ]:
df.head(5)

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,1/1/10,5.50901E+12,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,1/1/10,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,1/1/10,4.50308E+12,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,1/1/10,5.50901E+12,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,1/1/10,5.50901E+12,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96753 entries, 0 to 96752
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Recnum             96753 non-null  int64  
 1   Cardnum            96753 non-null  int64  
 2   Date               96753 non-null  object 
 3   Merchnum           93378 non-null  object 
 4   Merch description  96753 non-null  object 
 5   Merch state        95558 non-null  object 
 6   Merch zip          92097 non-null  float64
 7   Transtype          96753 non-null  object 
 8   Amount             96753 non-null  float64
 9   Fraud              96753 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 7.4+ MB


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
#remove all but P type transaction
df.drop(df.loc[df['Transtype']!='P'].index,inplace = True)

In [ ]:
#identify the single outlier in Amount
df.Amount.sort_values(ascending = False).head(5)

52714    3102045.53
47339      47900.00
59516      30372.46
80886      28392.84
89673      27218.00
Name: Amount, dtype: float64

In [ ]:
#remove outlier
df.drop(df.Amount.idxmax(),inplace = True)

In [ ]:
#check
df.Amount.sort_values(ascending = False).head(5)

47339    47900.00
59516    30372.46
80886    28392.84
89673    27218.00
1103     26910.00
Name: Amount, dtype: float64

In [ ]:
#sanity check
df.Transtype.unique()

array(['P'], dtype=object)

# Data Cleaning

**Missing Values** 

In [ ]:
df.isna().sum()

Recnum                  0
Cardnum                 0
Date                    0
Merchnum             3198
Merch description       0
Merch state          1020
Merch zip            4300
Transtype               0
Amount                  0
Fraud                   0
dtype: int64

* Merchnum

In [ ]:
#replace zeros with nan
df['Merchnum'] = df['Merchnum'].replace({'0':np.nan})

In [ ]:
#check if each merch description only has one corresponding merchnum
group = df.groupby(['Merch description']).Merchnum.nunique()
(group != 1).sum()

1329

In [ ]:
#create dictionary to store each merch description with corresponding merchnum
merch_merchnum = {}
for index, merch in df.loc[df['Merch description'].notnull() & df['Merchnum'].notnull(), 'Merch description'].items():
    if merch in merch_merchnum:
        merch_merchnum[merch].append(df.loc[index, 'Merchnum'])
    else:
        merch_merchnum[merch] = [df.loc[index, 'Merchnum']]



In [ ]:
#impute each merch description with the mode of corresponding merchnum
import statistics

for merch in merch_merchnum:
  merch_merchnum[merch] = statistics.mode(merch_merchnum[merch])

In [ ]:
#display first 5
list(merch_merchnum.items())[:5]
  

[('FEDEX SHP 12/23/09 AB#', '5.50901E+12'),
 ('SERVICE MERCHANDISE #81', '61003026333'),
 ('OFFICE DEPOT #191', '4.50308E+12'),
 ('FEDEX SHP 12/28/09 AB#', '5.50901E+12'),
 ('FEDEX SHP 12/22/09 AB#', '5.50901E+12')]

In [ ]:
#fill the merchnum with value of records share the same merch descriptions
df['Merchnum'] = df['Merchnum'].fillna(df['Merch description'].map(merch_merchnum))

In [ ]:
df['Merchnum'].isna().sum()

2094

In [ ]:
#convert adjustment transactions with unknown 
df.loc[df['Merch description'].str.contains('ADJUSTMENT'), 'Merchnum'] = 'unknown'

In [ ]:
df.loc[df['Merchnum'].isna(),'Merch description'].nunique()

504

In [ ]:
create_merchnum = {}
n = 1
for index,merch in df.loc[df['Merchnum'].isna(),'Merch description'].items():
  if merch not in create_merchnum:
    create_merchnum[merch] = n+ max(pd.to_numeric(df['Merchnum'],errors = 'coerce'))
    n+=1



In [ ]:
df['Merchnum'] = df['Merchnum'].fillna(df['Merch description'].map(create_merchnum))

In [ ]:
df['Merchnum'].isna().sum()

0

* Merch state

In [ ]:
df['Merch state'].isna().sum()

1020

In [ ]:
#transfer the zipcode to standard format
df['Merch zip'] = df['Merch zip'].astype('str')
df.loc[df['Merch zip'].str.contains('nan'), 'Merch zip'] = None
df['Merch zip'] = df['Merch zip'].apply(lambda x: x[:-2] if pd.notnull(x) else x)
df['Merch zip'].isna().sum()

4300

In [ ]:
us_zip = pd.read_csv('uszips.csv')

In [ ]:
us_zip.head(5)

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,601,18.18027,-66.75266,Adjuntas,PR,Puerto Rico,True,NaN,17126.0,102.6,72001,Adjuntas,"{""72001"": 98.73, ""72141"": 1.27}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,602,18.36075,-67.17541,Aguada,PR,Puerto Rico,True,NaN,37895.0,482.5,72003,Aguada,"{""72003"": 100}",Aguada,72003,False,False,America/Puerto_Rico
2,603,18.45744,-67.12225,Aguadilla,PR,Puerto Rico,True,NaN,49136.0,552.4,72005,Aguadilla,"{""72005"": 99.76, ""72099"": 0.24}",Aguadilla|Moca,72005|72099,False,False,America/Puerto_Rico
3,606,18.16585,-66.93716,Maricao,PR,Puerto Rico,True,NaN,5751.0,50.1,72093,Maricao,"{""72093"": 82.26, ""72153"": 11.68, ""72121"": 6.06}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,610,18.29110,-67.12243,Anasco,PR,Puerto Rico,True,NaN,26153.0,272.1,72011,Añasco,"{""72011"": 96.71, ""72099"": 2.81, ""72083"": 0.37,...",Añasco|Moca|Las Marías|Aguada,72011|72099|72083|72003,False,False,America/Puerto_Rico


In [ ]:
us_zip['zip'] = us_zip['zip'].astype('str')
us_zip = us_zip[['zip','state_id']]
us_zip.set_index('zip',inplace= True)
us_zip = us_zip['state_id'].to_dict()

In [ ]:
#impute the missing state with corresponding zipcode within U.S
df['Merch state'].fillna(df['Merch zip'].map(us_zip),inplace = True)
df['Merch state'].isna().sum()

980

In [ ]:
#create dictionary with merch description to corresponding state name
merch_state = {}
for index, merch in df.loc[df['Merch description'].notnull() & df['Merch state'].notnull(), 'Merch description'].items():
    if merch in merch_state:
        merch_state[merch].append(df.loc[index, 'Merch state'])
    else:
        merch_state[merch] = [df.loc[index, 'Merch state']]

#impute each merch description with the mode of corresponding mech description

for merch in merch_state:
  merch_state[merch] = statistics.mode(merch_state[merch])

In [ ]:
#impute the missing state with corresponding merch description
df['Merch state'].fillna(df['Merch description'].map(merch_state),inplace = True)
df['Merch state'].isna().sum()

323

In [ ]:
#create dictionary with merchnum to corresponding state name
merchnum_state = {}
for index, merch in df.loc[df['Merch state'].notnull(), 'Merchnum'].items():
    if merch in merchnum_state:
        merchnum_state[merch].append(df.loc[index, 'Merch state'])
    else:
        merchnum_state[merch] = [df.loc[index, 'Merch state']]

#impute each merch description with the mode of corresponding merchnum

for merch in merchnum_state:
  merchnum_state[merch] = statistics.mode(merchnum_state[merch])

In [ ]:
#impute impute the missing state with corresponding merchnum
df['Merch state'].fillna(df['Merchnum'].map(merchnum_state),inplace = True)
df['Merch state'].isna().sum()

314

In [ ]:
#set all the state name as foreign unknown for all the exsting zipcode that doesnt in US
for index,zip in df.loc[df['Merch state'].isnull(),'Merch zip'].items():
  if pd.notnull(zip):
    if zip not in us_zip.keys():
      df.loc[index,'Merch state'] = 'Foreign unknown'
df['Merch state'].isna().sum()

270

In [ ]:
#assign unknown state name to adjustment transaction
df.loc[df['Merch description'].str.contains('ADJUSTMENT'), 'Merch state'] = 'unknown'

In [ ]:
#fill the rest of merch state with unknowns
df['Merch state'].fillna('unknown',inplace = True)

In [ ]:
df['Merch state'].isna().sum()

0

* Merch Zip

In [ ]:
df['Merch zip'].isna().sum()

4300

In [ ]:
#create dictionary with merchnum corresponding merch zip
merchnum_zip = {}
for index, merch in df.loc[df['Merch zip'].notnull(), 'Merchnum'].items():
    if merch in merchnum_zip:
        merchnum_zip[merch].append(df.loc[index, 'Merch zip'])
    else:
       merchnum_zip[merch] = [df.loc[index, 'Merch zip']]

#impute each merch description with the mode of corresponding merchnum

for merch in merchnum_zip:
  merchnum_zip[merch] = statistics.mode(merchnum_zip[merch])

In [ ]:
#create dictionary with merch description corresponding merch zip
merch_zip = {}
for index, merch in df.loc[df['Merch zip'].notnull(), 'Merch description'].items():
    if merch in merch_zip:
        merch_zip[merch].append(df.loc[index, 'Merch zip'])
    else:
       merch_zip[merch] = [df.loc[index, 'Merch zip']]

#impute each merch description with the mode of corresponding merchnum

for merch in merch_zip:
  merch_zip[merch] = statistics.mode(merch_zip[merch])

In [ ]:
#create dictionary with state name corresponding merch zip
state_zip = {}
for index, merch in df.loc[df['Merch zip'].notnull(), 'Merch state'].items():
    if merch in state_zip:
        state_zip[merch].append(df.loc[index, 'Merch zip'])
    else:
        state_zip[merch] = [df.loc[index, 'Merch zip']]

#impute each merch description with the mode of corresponding merchnum

for merch in state_zip:
  state_zip[merch] = statistics.mode(state_zip[merch])

In [ ]:
#impute zipcode with mode of corresponding merhnum, merch description, and merch state
df['Merch zip'].fillna(df['Merchnum'].map(merchnum_zip),inplace = True)
df['Merch zip'].fillna(df['Merch description'].map(merch_zip),inplace = True)
df['Merch zip'].fillna(df['Merch state'].map(state_zip),inplace = True)
df['Merch zip'].isna().sum()

179

In [ ]:
 #assign unknown to adjustment transaction
df.loc[df['Merch description'].str.contains('ADJUSTMENT'), 'Merch zip'] = 'unknown'

In [ ]:
df['Merch zip'].fillna('unknown',inplace = True)
df['Merch zip'].isna().sum()

0

In [ ]:
#right alignment
df['Merch zip']=df['Merch zip'].apply(lambda x:'{0:0>5}'.format(x))

# Create Variables

* Benford's Law variable
